### Objective

In this notebook, we develop a local optimization routine to find the design that minimizes the weight while satisfying the geometrical and thermal constraints.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from two_sources import thermal_distribution_maxT

#### 1. Load initial guesses

In [2]:
Q1, Q2 = 300, 200
df_init = pd.read_csv(f'./dataset/{Q1}_{Q2}_proposal.csv')

In [3]:
# Filter the feasible solution
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)
Tjmax = 175
c_module, d_module = 61.4e-3, 106e-3

X = df_init.iloc[:, :-3].to_numpy()
T_max_list = []

for input in X:
    Tmax, _ = thermal_distribution_maxT(input, Data)
    T_max_list.append(Tmax)

df_init['Tj'] = np.array(T_max_list)
df_filtered = df_init[df_init['Tj']<=Tjmax].reset_index(drop=True)
print(f"Out of {df_init.shape[0]} initial guesses, {df_filtered.shape[0]} are valid!")
df_filtered = df_filtered.sort_values(by='utility', ascending=False).reset_index(drop=True)

Out of 50 initial guesses, 45 are valid!


#### 3. Problem definition

In [4]:
def objective(x):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Calculate weight
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(b / Fan_height)

    # Weight calculation
    w = density_Al*(b*d*L+n*(c*t*L))+ Fan_Weight*N_fan
    
    return w

In [5]:
def constraint_maxT(x, *args):
    
    # Unpack fixed parameters
    Q1, Q2, Data, Tmax_threshold = args
    
    # Calculate Tmax
    input = np.hstack((Q1, Q2, x))
    Tmax, _ = thermal_distribution_maxT(input, Data)
    
    # Constraint Tmax to be less than or equal to Tmax_threshold
    return Tmax_threshold - Tmax

In [6]:
def non_overlap_constraint(x, *args):

    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x
    
    # Unpack fixed parameters
    c_module, d_module = args
    
    # Calculate distances
    xc_dist = abs(xc1 - xc2)  # Assuming these are Xc_first and Xc_second
    yc_dist = abs(yc1 - yc2)  # Assuming these are Yc_first and Yc_second
    
    # Constraints for non-overlapping
    return max(xc_dist - c_module, yc_dist - d_module)

In [7]:
def t_upper_bound(x):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    return b / n - 1e-3 - t

In [8]:
def x1_upper_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Xc_max = b - c_module / 2

    return Xc_max - xc1

In [9]:
def x1_lower_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Xc_min = c_module / 2

    return xc1 - Xc_min

In [10]:
def x2_upper_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Xc_max = b - c_module / 2

    return Xc_max - xc2

In [11]:
def x2_lower_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Xc_min = c_module / 2

    return xc2 - Xc_min

In [12]:
def y1_upper_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Yc_max = L - d_module / 2

    return Yc_max - yc1

In [13]:
def y1_lower_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Yc_min = d_module / 2

    return yc1 - Yc_min

In [14]:
def y2_upper_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Yc_max = L - d_module / 2

    return Yc_max - yc2

In [15]:
def y2_lower_bound(x, *args):
    
    # Unpack design variables
    d, b, L, c, L_duct, n, t, xc1, yc1, xc2, yc2 = x

    # Unpack fixed parameters
    c_module, d_module = args

    Yc_min = d_module / 2

    return yc2 - Yc_min

In [16]:
class EvaluationCounter:
    def __init__(self, func):
        self.func = func
        self.counter = 0

    def __call__(self, *args):
        self.counter += 1
        return self.func(*args)

In [17]:
Tmax_contraint_counter = EvaluationCounter(constraint_maxT)

In [18]:
%%time

# Bounds
d_min, d_max = 5e-3, 30e-3
b_min, b_max = 73.7e-3, 307e-3
L_min, L_max = 127.2e-3, 530e-3
c_min, c_max = 10e-3, 39e-3
L_duct_min, L_duct_max = 20e-3, 50e-3
n_min, n_max = 10, 50

# Define bounds for each variable
bounds = [(d_min, d_max), (b_min, b_max), (L_min, L_max), (c_min, c_max), (L_duct_min, L_duct_max), (n_min, n_max),
          (1e-3, None),  # Placeholder for t, will need adjustment based on n and b
          (None, None), (None, None),  # Placeholder bounds for Xc_first, Yc_first
          (None, None), (None, None)]  # Placeholder bounds for Xc_second, Yc_second

# Initial guess
x0 = df_filtered.iloc[0, 2:-4].to_numpy()

# Optimization
result = minimize(objective, x0, method='SLSQP', bounds=bounds, 
                  constraints=[{'type': 'ineq', 'fun': Tmax_contraint_counter, 'args': (Q1, Q2, Data, Tjmax)},
                               {'type': 'ineq', 'fun': non_overlap_constraint, 'args': (c_module, d_module)},
                               {'type': 'ineq', 'fun': t_upper_bound},
                               {'type': 'ineq', 'fun': x1_upper_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': x1_lower_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': x2_upper_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': x2_lower_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': y1_upper_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': y1_lower_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': y2_upper_bound, 'args': (c_module, d_module)},
                              {'type': 'ineq', 'fun': y2_lower_bound, 'args': (c_module, d_module)}])

CPU times: total: 17.9 s
Wall time: 18 s


In [19]:
print(f"Tjmax constraints evaluation: {Tmax_contraint_counter.counter}")
print(f"Optimization results:")
columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'x0': x0, 'optimized': result.x})
df.index = columns
df

Tjmax constraints evaluation: 267
Optimization results:


,x0,optimized
d,0.005036,0.005000
b,0.135006,0.122800
L,0.130851,0.127200
c,0.017313,0.014257
L_duct,0.020918,0.020000
n,20.000000,13.119298
t,0.001064,0.001000
xc1,0.101107,0.092100
yc1,0.072332,0.063302
xc2,0.031513,0.030700


#### Verification

In [32]:
def verification_suite(design, Q1, Q2, Data, Tjmax, c_module, d_module):
    
    # Tmax constraint
    check = Tmax_contraint_counter(result.x, Q1, Q2, Data, Tjmax)
    if check >= 0:
        print(f"Tmax verification: True")
    else:
        print(f"Tmax exceeded by {check}")
    
    # Coordinates bounds
    check = x1_upper_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"x1 upper bound: True")
    else:
        print(f"x1 upper bound violated: {check}")

    check = x1_lower_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"x1 lower bound: True")
    else:
        print(f"x1 lower bound violated: {check}")

    check = x2_upper_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"x2 upper bound: True")
    else:
        print(f"x2 upper bound violated: {check}")

    check = x2_lower_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"x2 lower bound: True")
    else:
        print(f"x2 lower bound violated: {check}")

    check = y1_upper_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"y1 upper bound: True")
    else:
        print(f"y1 upper bound violated: {check}")

    check = y1_lower_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"y1 lower bound: True")
    else:
        print(f"y1 lower bound violated: {check}")

    check = y2_upper_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"y2 upper bound: True")
    else:
        print(f"y2 upper bound violated: {check}")

    check = y2_lower_bound(result.x, c_module, d_module)
    if check >= 0:
        print(f"y2 lower bound: True")
    else:
        print(f"y2 lower bound violated: {check}")

    
    # Non-overlapping bounds
    check = non_overlap_constraint(result.x, c_module, d_module)
    if check >= 0:
        print(f"Non-overlapping True")
    else:
        print(f"Non-overlapping violated: {check}")
    
    # t bounds
    check = t_upper_bound(result.x)
    if check >= 0:
        print(f"t bound verification True")
    else:
        print(f"t bound violated: {check}")

In [34]:
verification_suite(result.x, Q1, Q2, Data, Tjmax, c_module, d_module)

Tmax exceeded by -3.1693082291894825e-07
x1 upper bound: True
x1 lower bound: True
x2 upper bound: True
x2 lower bound violated: -7.476658181460039e-15
y1 upper bound: True
y1 lower bound: True
y2 upper bound: True
y2 lower bound violated: -1.1939060851062777e-12
Non-overlapping True
t bound verification True


#### Adjust n

In [45]:
import copy

# Extract optimized n
n = int(result.x[5])

# Check Tmax constraint
opt_design = copy.deepcopy(result.x)
opt_design[5] = n

input = np.hstack((Q1, Q2, opt_design))
Tmax, _ = thermal_distribution_maxT(input, Data)

if Tmax > Tjmax + 1e-6:
    print(f"Adjusting n from {result.x[5]} to {n} failed.")

    # Start adjustment
    while(Tmax > Tjmax + 1e-6):
        n += 1
        print(f"Instead, increase n to {n}")
    
        opt_design[5] = n
        input = np.hstack((Q1, Q2, opt_design))
        Tmax, _ = thermal_distribution_maxT(input, Data)

    print(f"Adjustment successed. New n ==> {n}")
    print(f"New Tmax ==> {Tmax}")
    print(f"Weight changed from {result.fun} to {objective(opt_design)}")

else:
    print(f"No change needed, {n} satisfy the constraint.")

Adjusting n from 13.119297673215913 to 13 failed.
Instead, increase n to 14
Adjustment successed. New n ==> 14
New Tmax ==> 170.86840909731754
Weight changed from 0.4783103033969633 to 0.4826226288921025
